In [ ]:
import pandas as pd
from sklearn import preprocessing
from sklearn import ensemble
import os


In [ ]:
TRAINING_DATA = os.environ.get("TRAINING_DATA")
TEST_DATA = os.environ.get("TEST_DATA")
FOLD = os.environ.get("FOLD")

TRAINING_DATA = '../input/train_folds.csv'
TEST_DATA = '../input/test.csv'
FOLD = 1


In [ ]:
FOLD_MAPPING = {
    0:[1,2,3,4],
    1:[0,2,3,4],
    2:[0,1,3,4],
    3:[0,1,2,4],
    4:[0,1,2,3]
}

In [ ]:
    df = pd.read_csv(TRAINING_DATA)
    train_df = df[df.kfold.isin(FOLD_MAPPING.get(FOLD))]
    valid_df = df[df.kfold==FOLD]

In [ ]:
ytrain = train_df.target.values
yvalid = valid_df.target.values

train_df = train_df.drop(['id','target','kfold'],axis=1)
valid_df = valid_df.drop(['id','target','kfold'],axis=1)

In [ ]:
valid_df = valid_df[train_df.columns]

In [ ]:
label_encoders = []
for c in train_df.columns:
    lbl = preprocessing.LabelEncoder()
    lbl.fit(train_df[c].values.tolist()+valid_df[c].values.tolist())
    train_df.loc[:,c] = lbl.transform(train_df[c].values.tolist())
    valid_df.loc[:,c] = lbl.transform(valid_df[c].values.tolist())
    label_encoders.append((c, lbl))

    

In [ ]:
clf = ensemble.RandomForestClassifier(n_jobs=-1, verbose=2)

In [ ]:
clf.fit(train_df, ytrain)

In [ ]:
preds = clf.predict_proba(valid_df)[:,1]

In [ ]:
preds